# Configuração do Ambiente

In [ ]:
import xml.etree.ElementTree as ET
import pandas as pd
import requests
import json

# Cotações

[Documentação da API](https://docs.awesomeapi.com.br/api-de-moedas)

## Funções

In [ ]:
def get_cotacoes_disponiveis() -> list[str]:
  '''
  Retorna uma lista com todas as cotações de
  moedas disponíveis.
  '''
  url = 'https://economia.awesomeapi.com.br/xml/available'
  dados = requests.get(url)
  # Cria uma árvore XML
  root = ET.fromstring(dados.text)
  # Guarda apenas as tags, que são as cotações disponíveis
  cotacoes_disponiveis = [child.tag for child in root]

  return cotacoes_disponiveis

In [ ]:
def get_cotacoes(*moedas:str) -> pd.DataFrame | None:
  '''
  Retorna um DataFrame com a cotação das moedas solicitadas.
  O Formato das moedas solicitadas deve ser do tipo: XXX-YYY.
  Ex: 'USD-BRL', 'EUR-BRL', 'BTC-BRL'
  Para cada moeda retorna:
    -> Máximo
    -> Mínimo
    -> Variação
    -> Porcentagem de Variação
    -> Compra
    -> Venda
  Caso não sejam passadas moedas, ou todas as passadas
  forem inválidas, a função retorna None.
  '''

  # Se não forem passadas moedas, retorn None
  if not moedas:
    return None

  # Lista as cotacoes disponiveis
  disponiveis = get_cotacoes_disponiveis()

  # Verifica se as moedas são válidas
  validas = list()
  n_validas = list()

  for moeda in moedas:
    if moeda in disponiveis:
      validas.append(moeda)
    else:
      n_validas.append(moeda)
  # Exibe as moedas válidas
  if n_validas:
    print('Cotações não encontradas:')
    for moeda in n_validas:
      print(moeda)
  # Se não houverem moedas válidas, retorna None
  if not validas:
    return None

  url = 'http://economia.awesomeapi.com.br/json/last/' #USD-BRL,EUR-BRL,ARS-BRL,BTC-BRL'
  for moeda in validas:
    url += moeda + ','
  # Remove a última vírgula adicionada
  url = url[:-1]
  # Faz a requisição dos dados e converter para DataFrame
  dados = requests.get(url)
  dados = dados.json()
  dados = pd.DataFrame(dados)
  # Guarda apenas as linhas necessárias e atualiza os índices e colunas
  cotacoes = dados[3:9]
  cotacoes.columns = [f'{i[:3]}->{i[3:]}' for i in cotacoes.columns]
  cotacoes.index = ['Máximo', 'Mínimo', 'Variação', 'Porcentagem de Variação', 'Compra', 'Venda']

  return cotacoes

## Uso

In [ ]:
get_cotacoes('USD-BRL', 'EUR-BRL', 'BTC-BRL', 'BRL-ARS')

,USD->BRL,EUR->BRL,BTC->BRL,BRL->ARS
Máximo,4.9682,5.3709,216987,167.0508
Mínimo,4.9471,5.3398,209280,166.2619
Variação,0.0114,0.0164,-1581,-0.5421
Porcentagem de Variação,0.23,0.31,-0.74,-0.32
Compra,4.9637,5.3652,210674,166.4092
Venda,4.9644,5.3732,210776,166.5078


# Índice IPCA

[Documentação da API](https://dadosabertos.bcb.gov.br/dataset/10844-indice-de-precos-ao-consumidor-amplo-ipca---servicos/resource/c0980df7-ad92-47af-b71c-790825f4710a)

## Funções

In [ ]:
def data_valida(data:str='') -> bool:
  '''
  Verifica se a data informada é valida segundo o
  padrão DD/MM/AAAA.
  retorna True se a data for válida e False caso contrário.
  '''
  # Verifica se o tamanho da string confere
  if len(data) != 10:
    return False
  # Verifica se os separadores estão corretos
  if data[2] != '/' or data[5] != '/':
    return False
  # Verifica se o dia é numérico e menor ou igual a 31
  dia = data[:2]
  if not dia.isnumeric() or int(dia) > 31:
    return False
  # Verifica se o mês é numérico e menor ou igual a 12
  mes = data[3:5]
  if not mes.isnumeric() or int(mes) > 12:
    return False
  # Verifica se o ano é numérico
  ano = data[6:]
  if not ano.isnumeric():
    return False

  return True

In [ ]:
def get_indices_IPCA(data_inicial:str='01/01/1992', data_final:str='01/01/2024') -> pd.DataFrame:

  data_inicial_default = '01/01/1992'
  data_final_default = '01/01/2024'
  url = 'https://api.bcb.gov.br/dados/serie/bcdata.sgs.10844/dados?formato=json'


  if data_valida(data_inicial):
    url += f'&dataInicial={data_inicial}'
  else:
    print('Data Inicial inválida')
    print(f'Usando data padrão: {data_inicial_default}')
    url += f'&dataInicial={data_inicial_default}'

  if data_valida(data_final):
    url += f'&dataFinal={data_final}'
  else:
    print('Data Final inválida')
    print(f'Usando data padrão: {data_final_default}')
    url += f'&dataFinal={data_final_default}'

  response = requests.get(url)
  indices = pd.DataFrame(response.json())

  return indices

## Uso

In [ ]:
get_indices_IPCA(data_inicial='01/01/2022', data_final='01/01/2023')

,data,valor
0,01/01/2022,0.39
1,01/02/2022,1.36
2,01/03/2022,0.45
3,01/04/2022,0.66
4,01/05/2022,0.85
5,01/06/2022,0.90
6,01/07/2022,0.80
7,01/08/2022,0.28
8,01/09/2022,0.40
9,01/10/2022,0.67


# Feriados

[Documentação da API](https://api.invertexto.com/api-feriados)

## Funções

In [ ]:
def get_feriados(ano:int=2024, estado:str='', token:str='') -> pd.DataFrame:
  '''
  Retorna um DataFrame com todos os feriados do período e
  região declarados.
  No parâmetro 'estado' deve ser passada apenas a sigla dele.
  Ex: estado='CE'
  Caso não seja passado nenhum estado, ou um estado inválido,
  serão retornados apenas os feriados nacionais.
  '''

  ufs = ['AC', 'AL', 'AP', 'AM', 'BA', 'CE', 'DF',
         'ES', 'GO', 'MA', 'MT', 'MS', 'MG', 'PA',
         'PB', 'PR', 'PE', 'PI', 'RJ', 'RN', 'RS',
         'RO', 'RR', 'SC', 'SP', 'SE', 'TO' ]

  colunas = ['data', 'nome', 'tipo', 'nível']

  if estado:
    if estado in ufs:
      estado = f'&state={estado}'
      colunas += ['lei']
    else:
      print('Estado inválido')
      estado = ''

  url = f'https://api.invertexto.com/v1/holidays/{ano}?token={token}{estado}'

  response = requests.get(url)
  feriados = response.json()
  feriados = pd.DataFrame(feriados).fillna('')

  feriados.columns = colunas
  # Ajusta a data, que está na forma AAAA-DD-MM
  feriados['data'] = feriados['data'].apply(lambda x: f'{x[5:]}-{x[:4]}')

  return feriados

## Uso

In [ ]:
get_feriados(ano=2024, estado='CE', token=token)

,data,nome,tipo,nível,lei
0,01-01-2024,Confraternização Universal,feriado,nacional,
1,02-12-2024,Carnaval,facultativo,nacional,
2,02-13-2024,Carnaval,facultativo,nacional,
3,02-14-2024,Quarta-feira de Cinzas,facultativo,nacional,
4,03-19-2024,Dia de São José,feriado,estadual,Lei federal nº 9.093/1995
5,03-25-2024,Abolição da escravidão no Ceará,feriado,estadual,"Art. 18, parágrafo único da constituição estadual"
6,03-29-2024,Sexta-feira Santa,facultativo,nacional,
7,04-21-2024,Tiradentes,feriado,nacional,
8,05-01-2024,Dia do Trabalhador,feriado,nacional,
9,05-30-2024,Corpus Christi,facultativo,nacional,
